In [1]:
import pandas as pd
import numpy as np

import bbg_symbology as bbs
from bbg_loader_core import *

In [2]:
def merge_dict(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

class ExpiredAliasService:
    def __init__(self, src=None):
        
        self._src = src
            
        if src is not None:
            self.expired = list(pd.read_csv(src, header=None)[1])
            
    def __contains__(self, key):
        return key in self.expired
    
    def __repr__(self):
        return '<{}: {}>'.format(self.__class__, str(self.expired))
    
    def add_expired(self, alias):
        self.expired.extend([alias])
        
    def save(self, fname=None):
        if fname is None:
            fname = self._src
            
        if fname is None:
            raise Error('No Filename Specified!')
            
        pd.Series(self.expired).to_csv(fname)

In [5]:
INPUT_PATH = '../_in/'

FUTURES_MONTHS = {m:i+1 for i,m in enumerate('FGHJKMNQUVXZ')}

FutChainRef = bbs.FuturesChainReference(INPUT_PATH + 'futures_historical_chain.csv')
BbgTckrService = bbs.BloombergTckrService()
FutAliasRef = bbs.FuturesAliasService(INPUT_PATH + 'fut_roots.csv')
ExpiredAliases = ExpiredAliasService('../_in/expired_aliases.csv')


x = FutAliasRef.bbg_to_alias_tckr('CLZ5 Comdty')
print(x)

cl.Z25


In [7]:
len(BbgTckrService.full_abb)

20585

In [11]:
expired_aliases = pd.Series(['cl.Z05', 'cl.Z06'])

In [13]:
expired_aliases.to_csv('../_in/expired_aliases.csv')


In [37]:

    
    

ExpiredAliases.add_expired('cl.Z09')
ExpiredAliases.save()

In [38]:
ExpiredAliases = ExpiredAliasService('../_in/expired_aliases.csv')
ExpiredAliases

<__main__.ExpiredAliasService: ['cl.Z05', 'cl.Z06', 'cl.Z09']>